<a href="https://colab.research.google.com/github/Leticiapp/SparkSample/blob/main/Spark_Session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

conf = SparkConf().set('spark.ui.port', '4050').setAppName("twitter").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)

In [9]:
import requests

url = "https://raw.githubusercontent.com/Leticiapp/SparkSample/main/data/tweets.csv"

response = requests.get(url)
open('tweets.csv', 'wb').write(response.content)

df = spark.read.option("inferSchema", True).option("header", True).csv('tweets.csv')
df.printSchema()

root
 |-- ItemID: integer (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- SentimentSource: string (nullable = true)
 |-- SentimentText: string (nullable = true)



In [10]:
df.write.saveAsTable('tweets')

In [11]:
spark.sql("select Sentiment, count(*) from tweets group by Sentiment").show()

+---------+--------+
|Sentiment|count(1)|
+---------+--------+
|      pos|   26921|
|        0|       2|
|      neg|   23079|
+---------+--------+



In [12]:
from pyspark.sql import HiveContext

hiveContext = HiveContext(sc)
hiveQuery = "select SentimentText from tweets where Sentiment='pos';"
dfPos = df = hiveContext.sql(hiveQuery)
rddPos = dfPos.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:733: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [29]:
stopWords = ['i', 'me', 'my', 'myself', 'we', 'get', 'this', 'do', 'up', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'she', 'her', 'the', 'to', 'a', 'and', 'it', 'for', 'is', 'in', 'with', 'was', 'u', 'will', 'not', 'all', 'at', 'of', 'that', 'on', 'have', 'so', 'be', 'just', 'are', 'but']

In [31]:
contadorPalavras = rddPos.map(lambda x:x.SentimentText.replace(',','').replace('.','').replace('-','').lower()) \
.flatMap(lambda x: x.split()) \
.filter(lambda x: x not in stopWords) \
.map(lambda x: (x, 1)) \
.reduceByKey(lambda x, y: x + y) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False)

In [32]:
contadorPalavras.take(10)

[(1802, 'good'),
 (1793, "i'm"),
 (1633, 'love'),
 (1408, 'like'),
 (1216, 'thanks'),
 (1166, 'lol'),
 (1092, "it's"),
 (1026, 'can'),
 (1024, 'know'),
 (1007, 'now')]